In [5]:
#✅ Features
# Works for all common vector formats: .shp, .gpkg, .geojson, .json, .kml, .gml.
# Uses parallel processing to speed up large batches.
# Saves each file to Excel in the same folder, with each field as a separate sheet.
# Counts are sorted descending.
# Sheet names automatically truncated to 31 characters for Excel compatibility.

# Install all necessary packages
!pip install geopandas pandas openpyxl fiona shapely pyproj rtree


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import os
# CHANGE THIS: Path to the folder in your Google Drive containing your vector files
# Example: 'My Drive/Shapefiles'
# CHANGE THIS: Path to the folder in your Google Drive containing your vector files
folder_path = '/content/drive/My Drive/Shapefiles' # @param {type:"string"}

if not os.path.exists(folder_path):
    raise SystemExit("Folder not found! Please check your folder path.")
else:
    print(f"Processing vector files in: {folder_path}")

# Create output folder for Excel files
output_folder = os.path.join(folder_path, "Excel_Outputs")
os.makedirs(output_folder, exist_ok=True)
print(f"Excel outputs will be saved in: {output_folder}")

# Prompt user for fields to count
print("Enter the attribute fields to count (case-sensitive), separated by commas.")
print("Or type ALL (case-sensitive) to count all fields in each file.")

user_input = input("Fields to count: ").strip()

if user_input == "ALL":
    fields_to_count = "ALL"
else:
    fields_to_count = [f.strip() for f in user_input.split(",")]

print(f"Fields to count: {fields_to_count}")




Processing vector files in: /content/drive/My Drive/Shapefiles
Excel outputs will be saved in: /content/drive/My Drive/Shapefiles/Excel_Outputs
Enter the attribute fields to count (case-sensitive), separated by commas.
Or type ALL (case-sensitive) to count all fields in each file.
Fields to count: ownership, municipality
Fields to count: ['ownership', 'municipality']


In [18]:
import glob
import pandas as pd
import geopandas as gpd
from concurrent.futures import ThreadPoolExecutor

# Supported vector file extensions
extensions = ['*.shp', '*.gpkg', '*.geojson', '*.json', '*.kml', '*.gml']

# Find all vector files
vector_files = []
for ext in extensions:
    vector_files.extend(glob.glob(os.path.join(folder_path, ext)))

if not vector_files:
    raise SystemExit("No vector files found in the folder.")

print(f"Found {len(vector_files)} vector files.")

# Function to process a single vector file
def process_vector_file(file_path, output_folder, fields_to_count="ALL"):
    try:
        gdf = gpd.read_file(file_path)
    except Exception as e:
        print(f"Failed to read {file_path}: {e}")
        return

    layer_name = os.path.splitext(os.path.basename(file_path))[0]
    sheets = {}

    # Determine which fields to process
    if fields_to_count == "ALL":
        fields = gdf.columns
    else:
        fields = [f for f in fields_to_count if f in gdf.columns]
        if not fields:
            print(f"No matching fields in {file_path}. Skipping.")
            return

    # Count and save for each selected field
    for field in fields:
        counts = gdf[field].value_counts(dropna=False).sort_values(ascending=False)
        df = pd.DataFrame({field: counts.index, 'Count': counts.values})
        sheets[field[:31]] = df  # truncate sheet name to 31 chars

    # Output Excel file path
    output_file = os.path.join(output_folder, f"{layer_name}_attribute_counts.xlsx")
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        for sheet_name, df in sheets.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Saved: {output_file}")

# Parallel processing
max_threads = min(8, len(vector_files))  # adjust threads as needed
with ThreadPoolExecutor(max_workers=max_threads) as executor:
    for file_path in vector_files:
        executor.submit(process_vector_file, file_path, output_folder, fields_to_count)

print("Batch processing complete! All Excel files saved in Excel_Outputs folder.")


Found 8 vector files.
Saved: /content/drive/My Drive/Shapefiles/Excel_Outputs/NL_airports_attribute_counts.xlsx
Saved: /content/drive/My Drive/Shapefiles/Excel_Outputs/NL_dams_attribute_counts.xlsx
Saved: /content/drive/My Drive/Shapefiles/Excel_Outputs/NL_agriculture_attribute_counts.xlsx
Saved: /content/drive/My Drive/Shapefiles/Excel_Outputs/NL_coastal_marine_infrastructure_attribute_counts.xlsx
Saved: /content/drive/My Drive/Shapefiles/Excel_Outputs/NL_parks_attribute_counts.xlsx
Saved: /content/drive/My Drive/Shapefiles/Excel_Outputs/NL_parking_attribute_counts.xlsx
Saved: /content/drive/My Drive/Shapefiles/Excel_Outputs/NL_roads_attribute_counts.xlsx
Saved: /content/drive/My Drive/Shapefiles/Excel_Outputs/NL_buildings_attribute_counts.xlsx
Batch processing complete! All Excel files saved in Excel_Outputs folder.
